In [1]:
ISSUE=""
EMBEDDER_MODEL_NAME = 'nomic-embed-text:latest'
LLM_MODEL_NAME = 'llama3.2:latest'
REPO = '.'

## Embeddings

In [2]:
from embeddings.ollama import OllamaEmbedder

embeedder = OllamaEmbedder(model=EMBEDDER_MODEL_NAME)

ModuleNotFoundError: No module named 'embeddings'

## Vectorize

In [ ]:
from embeddings.vectorizers.base import Vectorizer
from embeddings.vectorizers.chunk_vectorizer import ChunkVectorizer

vectorizer = ChunkVectorizer(embedder=embeedder)
Vectorizer(vectorizer).vectorize_to_database(None, repo_destination=REPO)

## Embeddings:lookup

In [ ]:
embeddings_results = Embedder(OllamaEmbedder, EMBEDDER_MODEL_NAME).retrieve_embeddings(ISSUE)

similar_embeddings = [
        (embedding.repository, embedding.file_path, embedding.text) for embedding in embeddings_results
    ]

## Prompt

In [ ]:
import json
from llm import get_prompt, prepare_context


prompt = get_prompt(ISSUE)
prepared_context = prepare_context(similar_embeddings, prompt)

## LLM

In [ ]:
from litellm_service.llm_requester import Requester
from litellm_service.ollama import OllamaRequester
from llm import validate_llm_response


requester = Requester(OllamaRequester, model=LLM_MODEL_NAME)
llm_response = requester.completion_without_proxy(prepared_context)
redo, redo_reason = validate_llm_response(llm_response)

In [ ]:
llm_response